# 4.1-toxvaldb-results-analysis
The distribution of ROC-AUC values for the toxvaldb prediction pipeline is unusual. There are some assays with a ROC AUC score of 0, NaN, and 1. I need to take a further look at the scoring files as well as the composition of the assays themselves.

## TL;DR
The unusual scores result from the small testing set sizes. I have included three examples below to explain cases where ROC AUC scores are 1, 0, and NaN.

## Quick prediction score check

In [1]:
import os
import logging
import click
import pandas as pd
from rdkit import Chem
from rdkit import RDLogger
import selfies as sf
import numpy as np
import duckdb

In [2]:
folder_path = "../data/processed/scores/"

# Get a list of all Parquet files in the folder
parquet_files = [f for f in os.listdir(folder_path) if f.endswith('.parquet')]

# Initialize an empty list to hold dataframes
df_list = []

for file in parquet_files:
    temp_df = pd.read_parquet(os.path.join(folder_path, file))
    temp_df['filename'] = os.path.basename(file)
    df_list.append(temp_df)

# Read in all Parquet files and concatenate them into a single DataFrame
df = pd.concat(df_list)

# Add a new column with the characters between the 1st and 3rd underscores of the basename
df['assay'] = df['filename'].apply(lambda x: '_'.join(x.split('_')[1:3]))

In [3]:
df.head(15)

,feature,accuracy,precision,recall,f1,auc_roc,filename,auc_pr,assay
0,1024,1.000000,0.000000,0.000000,0.000000,NaN,score_assay_56_toxvaldb_2023_logistic_regressi...,NaN,assay_56
0,1024,0.714286,0.687500,0.785714,0.733333,0.841837,score_assay_61_toxvaldb_2023_logistic_regressi...,0.851535,assay_61
0,1024,0.500000,0.000000,0.000000,0.000000,0.500000,score_assay_72_toxvaldb_2023_logistic_regressi...,0.583333,assay_72
0,1024,0.714286,0.666667,0.666667,0.666667,0.583333,score_assay_89_toxvaldb_2023_logistic_regressi...,0.555556,assay_89
0,1024,0.750000,0.500000,1.000000,0.666667,1.000000,score_assay_45_toxvaldb_2023_logistic_regressi...,1.000000,assay_45
0,1024,0.677419,0.588235,0.769231,0.666667,0.756410,score_assay_38_toxvaldb_2023_logistic_regressi...,0.709669,assay_38
0,1024,0.444444,0.500000,0.400000,0.444444,0.575000,score_assay_6_toxvaldb_2023_logistic_regressio...,0.703077,assay_6
0,1024,0.750000,0.000000,0.000000,0.000000,1.000000,score_assay_63_toxvaldb_2023_logistic_regressi...,1.000000,assay_63
0,1024,0.375000,0.600000,0.272727,0.375000,0.600000,score_assay_29_toxvaldb_2023_logistic_regressi...,0.754195,assay_29
0,1024,0.285714,0.000000,0.000000,0.000000,0.000000,score_assay_54_toxvaldb_2023_logistic_regressi...,0.320635,assay_54


We can see that there are a few strange sets of results here:
- assay_56: it has an accuracy of 1, but values of 0 for precision, recall, and f1, and NaN for auc-roc.
- assay_45: accuracy of 0.75, but auc-roc of 1
- assay_54: accuracy of 0.29, but auc-roc of 0

I will dig further into each of these assays to try and work out what has gone wrong.

## assay_56
I want to start by having a look at the predictions that were made for this assay.

In [4]:
assay_56_preds = pd.read_parquet("/Users/sethhowes/Desktop/FS-Tox/data/processed/predictions/assay_56_toxvaldb_2023_logistic_regression_ecfp4_1024.parquet")
assay_56_preds.head(10)

,canonical_smiles,preds,preds_proba,ground_truth,model
8,CCOCC,0,0.241170,0,logistic_regression
10,COc1cc(-c2ccc(/N=N/c3ccc4c(S(=O)(=O)[O-])cc(S(...,0,0.032146,0,logistic_regression
11,Cc1cc(-c2ccc(N=Nc3c(S(=O)(=O)O)cc4cc(S(=O)(=O)...,0,0.047591,0,logistic_regression
16,CC(Cl)(Cl)Cl,0,0.238468,0,logistic_regression


So it appears that there is a very small test set consisting of only 6 values. Each of these values is classed as non-toxic. I want to see if there is an error in the toxicity binarisation process whereby all of the values are being classed as non-toxic.

In [5]:
assay_56 = pd.read_parquet("/Users/sethhowes/Desktop/FS-Tox/data/processed/assays/assay_56_toxvaldb_2023.parquet")
assay_56["ground_truth"].value_counts()

0    21
1     4
Name: ground_truth, dtype: int64

So there are only 4 values that are being classed as toxic. I have a feeling that this could be due to there only being two different numeric outcomes for observations in this assay.

In order to read the original numeric values prior to binarisation, I will have to repeat the processing steps carried out in `src/data/raw_to_assays.py`.

In [6]:
# Define a function to convert InChI to SMILES
def convert_to_smiles(inchi):

    # Suppress RDKit warnings
    RDLogger.DisableLog("rdApp.*")
    
    mol = Chem.MolFromInchi(inchi)
    if mol is None:
        return 'InvalidInChI'  # Placeholder for invalid InChI
    smiles = Chem.MolToSmiles(mol)
    return smiles

In [7]:
# Create list of columns to include
def get_assays(input_filepath, identifier):

    vars_to_extract = [
        "dtxsid",
        "casrn",
        "long_ref",
        "toxval_type",
        "common_name",
        "exposure_route",
        "toxval_units",
        "study_type",
        "source",
        "toxval_numeric",
        "toxval_numeric_qualifier",
    ]

    df = pd.read_csv(input_filepath, usecols=vars_to_extract)

    # Replace '-' with np.nan
    df.replace("-", np.nan, inplace=True)

    assay_components = [
        "long_ref",
        "toxval_type",
        "common_name",
        "exposure_route",
        "toxval_units",
        "study_type",
        "source",
    ]

    # Drop rows with null values for key variables
    df.dropna(subset=assay_components + ["toxval_numeric"], inplace=True)

    # Drop long refs with different NA formats
    na_list = ["- - - NA", "- - - -", "- Unnamed - NA", "Unknown", "- Unnamed - -"]
    df = df[~df["long_ref"].isin(na_list)]

    # Remove those records with toxval_numeric_qualifier not equal to "="
    df = df[df["toxval_numeric_qualifier"] != "="]

    # Drop the toxval_numeric_qualifier column as it is no longer needed
    df.drop(columns="toxval_numeric_qualifier", inplace=True)

    # Read in the identifiers
    identifiers = pd.read_csv(identifier)

    # Merge tox data with the molecule identifiers
    df_with_inchis = df.merge(identifiers, how="left", on="dtxsid")

    # Apply the function to each value in the Series
    smiles_series = df_with_inchis["inchi"].astype(str).apply(convert_to_smiles)

    # Reset index to ensure smiles series appends correctly
    df = df.reset_index(drop=True)

    # Add the smiles column to the DataFrame
    df["smiles"] = smiles_series

    # Drop rows where smiles column is equal to 'InvalidInChI'
    df = df[df["smiles"] != "InvalidInChI"]

    # Get records that belong to a group of greater than 10 members
    df = df.groupby(assay_components).filter(lambda x: len(x) >= 24)

    # Replace all '_' with '-' in long_ref column
    df["long_ref"] = df["long_ref"].str.replace("_", "-")

    # Create a new column that is a combination of the assay_components
    df["combined"] = df[assay_components].apply(
        lambda row: "_".join(row.values.astype(str)), axis=1
    )

    # Create a pivot table where each unique combination forms a separate column
    df_pivoted = df.pivot_table(
        index="smiles",
        columns="combined",
        values="toxval_numeric",
        aggfunc=np.mean,
    )

    return df_pivoted

In [8]:
input_filepath = "/Users/sethhowes/Desktop/FS-Tox/data/raw/toxvaldb_2023.csv"
identifier = "/Users/sethhowes/Desktop/FS-Tox/data/external/DSSTox_Identifiers_and_CASRN_2021r1.csv"

assays = get_assays(input_filepath, identifier)

In [9]:
assay_56_colname = assays.columns[assays.columns.str.startswith("Jpn. J. Toxicol. Environ. Health32(1):")]
assay_56_raw = assays[assay_56_colname]
assay_56_raw.dropna(inplace=True)

/var/folders/t1/p54ybn1n4j3_ywd21t7skv2m0000gn/T/ipykernel_55656/842036901.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assay_56_raw.dropna(inplace=True)


In [12]:
assay_56_raw.value_counts()

Jpn. J. Toxicol. Environ. Health32(1): 46-53 Tsuji,S., Y. Tonogai, Y. Ito, and S. Kanoh The Influence of Rearing Temperatures on the Toxicity of Various Environmental Pollutants for Killifish (Oryzias latipes) 1986_LC50_Japanese Medaka_aqueous_mg/m3_mortality_ECOTOX
1000000.0                                                                                                                                                                                                                                                                     21
10000000.0                                                                                                                                                                                                                                                                     4
dtype: int64

It is clear that the reason there is an imbalance when it should be a 50/50 split is that there are only two values. I must therefore remove those assays for which there are only 2 values.

So in conclusion, the reason that we are getting the strange evaluation metrics for this assay is that there are no positive cases in the test set. As such, the numerators of the equations for calculating precision and recall are both 0. Additionally, the aur-roc curve will have a true positive rate of 0, as there are no true positives.

Accuracy is therefore the only metric that accounts for true negative values, hence why it is positive whilst the rest are negative.

## assay_45
This assay has the following scores: accuracy of 0.75, but auc-roc of 1.

As I did previously, I will start off by checking the raw predictions.

In [15]:
assay_45_preds = pd.read_parquet("/Users/sethhowes/Desktop/FS-Tox/data/processed/predictions/assay_45_toxvaldb_2023_logistic_regression_ecfp4_1024.parquet")
assay_45_preds

,canonical_smiles,preds,preds_proba,ground_truth,model
5,CCCCCC[C@H](C/C=C\CCCCCCCC(=O)OCCOC)OC(C)=O,1,0.942744,1,logistic_regression
15,C1COCCN1.Cl,0,0.101234,0,logistic_regression
19,COC(=O)C1=C(C)NC(C)=C(C(=O)OCCN(C)Cc2ccccc2)C1...,0,0.495211,0,logistic_regression
20,COCCCO,1,0.876955,0,logistic_regression


The accuracy is 0.75 as the 3/4 predictions are correct.

I will now calculate the roc-auc score.

In [18]:
from sklearn.metrics import roc_auc_score

# Define the predicted probabilities and actual labels
predicted_probs = assay_45_preds["preds_proba"]
actual_labels = assay_45_preds["ground_truth"]

# Calculate ROC AUC score using roc_auc_score()
roc_auc = roc_auc_score(actual_labels, predicted_probs)

# Print the ROC AUC score
print("ROC AUC score:", roc_auc)

ROC AUC score: 1.0


ROC AUC is a rank-based metric. It is calculated by constructing the ROC curve for n-1 thresholds, where n is the number of predictions made (in this case 4). For every threshold calculated, the true positive rate will be 1, as there is only one positive prediction made by the model. Therefore for all values of the false positive rate, the true positive rate will always be 1, resulting in a ROC AUC score of 1.

## assay_54
A reminder of the scores for this assay - accuracy of 0.29, but auc-roc of 0.

As I did previously, I want to check the predictions made for this assay.

In [23]:
assay_54_preds = pd.read_parquet("/Users/sethhowes/Desktop/FS-Tox/data/processed/predictions/assay_54_toxvaldb_2023_logistic_regression_ecfp4_1024.parquet")
assay_54_preds

,canonical_smiles,preds,preds_proba,ground_truth,model
4,CNC(=N)S,0,0.435951,0,logistic_regression
13,Sc1nc2ccc(Cl)cc2[nH]1,0,0.472622,0,logistic_regression
16,C=CCN=C(S)Nc1ccccc1,0,0.100480,1,logistic_regression
19,CCCCCCCCN(C)CCCCCCCC,1,0.679625,0,logistic_regression
20,CN1C2CCC(C)(C)C1CC(OC(=O)C(O)(c1cccs1)c1cccs1)C2,1,0.537331,0,logistic_regression
21,SC(=NNc1ccccc1)NNc1ccccc1,0,0.182318,1,logistic_regression
28,SC1=NCCS1,0,0.343517,1,logistic_regression


We can again see how these outputs now make sense. The model is correctly classifying 2/7 observations, hence an accuracy of 0.29.

However, it is not correctly classifying any true positives, which means that the true positive rate is 0, and therefore the ROC AUC score is 1.